<a href="https://colab.research.google.com/github/madhura2024/myAI_MODELS/blob/main/sentiment_analyis_summarization_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

STARTING DEEP LEARNING ( making sentiment analysis and summarization using deep learning )




   2 TYPES OF CLEANING ----------------->>>>>>>>>>    
1. structural ( drop , duplicates , strip , dummy etc )
2. textual ( sub , lower etc )





In [1]:
import pandas as pd

model = pd.read_csv("https://zenodo.org/record/10157504/files/Musical_instruments_reviews2.csv")
# print(model.columns)
model = model.head(10)
model = model.dropna(subset=['reviewText']).drop_duplicates(subset=['reviewText'])
# model------------>>>>>>>> is your whole table.model.               ['text'] --------------->>>>>>>>>Just one column from the table — the part with your actual sentences or words.
model['reviewText'] = model['reviewText'].astype(str).str.strip()


In [2]:
import re  # Get the magic toolbox for finding and replacing stuff in text
import nltk  # Bring in the big NLP helper library
from nltk.corpus import stopwords  # Grab the list of boring common words like "the" and "and"

nltk.download('stopwords')  # Download that boring word list once so we can use it
stop_words = set(stopwords.words("english"))  # Put those boring words in a fast-check list

def clean_text(text):
    text = text.lower()  # Make all letters small so "Hello" and "hello" look the same
    text = re.sub(r"http\S+|www\S+|<.*?>|[^a-z\s]", " ", text)  # Trash links, weird tags, and weird symbols with a space
    text = re.sub(r"\s+", " ", text).strip()  # Squish many spaces into one and wipe spaces at the ends
    return " ".join(word for word in text.split() if word not in stop_words)  # Kick out boring words and glue what's left back together

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


to check sentiment we have a column called overall where there are stars. think of it as a column array . use for loop and if else to dtermine mood.

In [3]:
sentiments = []

for i in range(len(model['overall'])):
    rating = model['overall'][i]
    if rating >= 4:
        sentiments.append("happy")
    elif rating <= 2:
        sentiments.append("sad")
    else:
        sentiments.append("neutral")

model['sentiment'] = sentiments


call

In [4]:
model['clean_review'] = model['reviewText'].apply(clean_text)

sentiment

In [5]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

sentiment_results = []

for text in model['reviewText']:
    result = sentiment_pipeline(text[:512])
    sentiment_results.append(result[0]['label'])

model['sentiment'] = sentiment_results


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


summarize

In [6]:
summarizer = pipeline("summarization")

summaries = []

for text in model['reviewText']:
    if len(text) > 20:
        result = summarizer(text[:1000])
        summaries.append(result[0]['summary_text'])
    else:
        summaries.append("Too short to summarize")

model['summary'] = summaries

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Device set to use cpu
Your max_length is set to 142, but your input_length is only 61. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)
Your max_length is set to 142, but your input_length is only 124. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=62)
Your max_length is set to 142, but your input_length is only 87. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)
Your max_length is set to 142, but your input_length is only 45. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('..

time taking so check just a few

In [7]:
print(model[['reviewText', 'sentiment', 'summary']].head(10))

                                          reviewText sentiment  \
0  Not much to write about here, but it does exac...  POSITIVE   
1  The product does exactly as it should and is q...  POSITIVE   
2  The primary job of this device is to block the...  NEGATIVE   
3  Nice windscreen protects my MXL mic and preven...  NEGATIVE   
4  This pop filter is great. It looks and perform...  POSITIVE   
5  So good that I bought another one.  Love the h...  POSITIVE   
6  I have used monster cables for years, and with...  POSITIVE   
7  I now use this cable to run from the output of...  NEGATIVE   
8  Perfect for my Epiphone Sheraton II.  Monster ...  POSITIVE   
9  Monster makes the best cables and a lifetime w...  POSITIVE   

                                             summary  
0   It is one of the lowest prices pop filters on...  
1   The product does exactly as it should and is ...  
2   The device blocks the breath that would other...  
3   Nice windscreen protects my MXL mic and preve... 